# Customer.IO Device Management

This notebook demonstrates how to manage user devices in Customer.IO for push notifications.
Device management is essential for mobile applications that need to send push notifications to iOS and Android devices.

## Setup and Imports

Import the necessary functions and initialize the Customer.IO client.

In [ ]:
# Setup and imports
import os
from datetime import datetime
from utils.api_client import CustomerIOClient
from utils.people_manager import identify_user
from utils.device_manager import (
    register_device,
    update_device,
    delete_device
)
from utils.exceptions import CustomerIOError, ValidationError

# Initialize client
API_KEY = os.getenv('CUSTOMERIO_API_KEY', 'your-api-key-here')
client = CustomerIOClient(api_key=API_KEY, region='us')

print("Customer.IO device management functions loaded successfully")

## Understanding Device Management

Devices in Customer.IO represent:
- Mobile devices (iOS/Android) for push notifications
- Browser instances for web push
- Any endpoint that can receive notifications

Key concepts:
- **Device Token**: Unique identifier from push notification service (APNs/FCM)
- **Device Type**: Platform identifier ("ios" or "android")
- **Metadata**: Additional device information (OS version, app version, etc.)

## Registering Devices

Register a device when a user grants push notification permissions.

In [ ]:
# Example: Register an iOS device
ios_device_token = "a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6q7r8s9t0"
user_id = "user_ios_001"

# First, ensure the user exists in Customer.IO
try:
    identify_user(
        client=client,
        user_id=user_id,
        traits={"email": f"{user_id}@example.com", "name": "iOS User"}
    )
    print(f"User {user_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying user: {e}")

try:
    result = register_device(
        client=client,
        user_id=user_id,
        device_token=ios_device_token,
        device_type="ios"
    )
    print("iOS device registered successfully")
    print(f"Result: {result}")
except CustomerIOError as e:
    print(f"Error registering iOS device: {e}")

In [ ]:
# Example: Register an Android device with metadata
android_device_token = "fCm_token_example_123456789"
android_user_id = "user_android_001"
device_metadata = {
    "app_version": "2.5.1",
    "os_version": "Android 13",
    "manufacturer": "Samsung",
    "model": "Galaxy S23",
    "language": "en-US",
    "timezone": "America/New_York"
}

# First, ensure the user exists in Customer.IO
try:
    identify_user(
        client=client,
        user_id=android_user_id,
        traits={"email": f"{android_user_id}@example.com", "name": "Android User"}
    )
    print(f"User {android_user_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying user: {e}")

try:
    result = register_device(
        client=client,
        user_id=android_user_id,
        device_token=android_device_token,
        device_type="android",
        metadata=device_metadata
    )
    print("Android device registered with metadata")
except CustomerIOError as e:
    print(f"Error registering Android device: {e}")

## Updating Device Information

Update device metadata when app versions change or settings are modified.

In [ ]:
# Update device with new app version
updated_metadata = {
    "app_version": "2.6.0",
    "last_opened": datetime.now().isoformat(),
    "notification_enabled": True,
    "sound_enabled": True,
    "badge_enabled": True
}

try:
    result = update_device(
        client=client,
        user_id=android_user_id,
        device_token=android_device_token,
        device_type="android",
        metadata=updated_metadata
    )
    print("Device updated with new app version")
except CustomerIOError as e:
    print(f"Error updating device: {e}")

In [ ]:
# Update device preferences
preference_metadata = {
    "push_categories": ["promotions", "updates", "alerts"],
    "quiet_hours_start": "22:00",
    "quiet_hours_end": "08:00",
    "preferred_language": "es-ES"
}

try:
    result = update_device(
        client=client,
        user_id="user_ios_001",
        device_token=ios_device_token,
        device_type="ios",
        metadata=preference_metadata,
        timestamp=datetime.now()
    )
    print("Device preferences updated")
except CustomerIOError as e:
    print(f"Error updating preferences: {e}")

## Multi-Device Management

Handle users with multiple devices (phone, tablet, etc.).

In [ ]:
# Register multiple devices for one user
user_id = "user_multi_001"
devices = [
    {
        "token": "iphone_token_abc123",
        "type": "ios",
        "metadata": {"device_name": "John's iPhone", "model": "iPhone 14 Pro"}
    },
    {
        "token": "ipad_token_def456",
        "type": "ios",
        "metadata": {"device_name": "John's iPad", "model": "iPad Air"}
    },
    {
        "token": "android_work_token_789",
        "type": "android",
        "metadata": {"device_name": "Work Phone", "model": "Pixel 7"}
    }
]

# First, ensure the user exists in Customer.IO
try:
    identify_user(
        client=client,
        user_id=user_id,
        traits={"email": f"{user_id}@example.com", "name": "Multi Device User"}
    )
    print(f"User {user_id} identified successfully")
except CustomerIOError as e:
    print(f"Error identifying user: {e}")

for device in devices:
    try:
        register_device(
            client=client,
            user_id=user_id,
            device_token=device["token"],
            device_type=device["type"],
            metadata=device["metadata"]
        )
        print(f"Registered: {device['metadata']['device_name']}")
    except CustomerIOError as e:
        print(f"Error registering {device['metadata']['device_name']}: {e}")

## Deleting Devices

Remove devices when users log out, uninstall the app, or revoke permissions.

In [ ]:
# Delete a device when user logs out
try:
    result = delete_device(
        client=client,
        user_id="user_ios_001",
        device_token=ios_device_token,
        device_type="ios"
    )
    print("Device removed successfully")
except CustomerIOError as e:
    print(f"Error deleting device: {e}")

In [ ]:
# Delete device with timestamp for audit trail
deletion_time = datetime.now()

try:
    result = delete_device(
        client=client,
        user_id="user_android_001",
        device_token=android_device_token,
        device_type="android",
        timestamp=deletion_time
    )
    print(f"Device deleted at {deletion_time.isoformat()}")
except CustomerIOError as e:
    print(f"Error deleting device: {e}")

## Real-World Example: Mobile App Lifecycle

Complete device management workflow for a mobile application.

In [ ]:
class DeviceManager:
    """Helper class for device lifecycle management."""
    
    def __init__(self, client):
        self.client = client
    
    def on_app_launch(self, user_id, device_token, device_type, app_version):
        """Called when app launches - update device info."""
        metadata = {
            "app_version": app_version,
            "last_active": datetime.now().isoformat(),
            "session_count": 1  # In real app, increment existing count
        }
        
        try:
            register_device(
                self.client,
                user_id,
                device_token,
                device_type,
                metadata
            )
            print(f"Device active: {device_type} for user {user_id}")
            return True
        except CustomerIOError as e:
            print(f"Failed to update device: {e}")
            return False
    
    def on_permission_granted(self, user_id, device_token, device_type, device_info):
        """Called when user grants push permission."""
        metadata = {
            **device_info,
            "permission_granted_at": datetime.now().isoformat(),
            "notification_enabled": True
        }
        
        try:
            register_device(
                self.client,
                user_id,
                device_token,
                device_type,
                metadata
            )
            print("Push notifications enabled")
            return True
        except CustomerIOError as e:
            print(f"Failed to enable push: {e}")
            return False
    
    def on_logout(self, user_id, device_token, device_type):
        """Called when user logs out."""
        try:
            delete_device(
                self.client,
                user_id,
                device_token,
                device_type
            )
            print("Device removed on logout")
            return True
        except CustomerIOError as e:
            print(f"Failed to remove device: {e}")
            return False

In [ ]:
# Example usage of DeviceManager
device_mgr = DeviceManager(client)

# Simulate app lifecycle
user_id = "user_lifecycle_001"
device_token = "lifecycle_token_123"
device_type = "ios"

# 1. App launch
device_mgr.on_app_launch(user_id, device_token, device_type, "3.0.0")

# 2. User grants permission
device_info = {
    "model": "iPhone 15",
    "os_version": "iOS 17.0",
    "language": "en"
}
device_mgr.on_permission_granted(user_id, device_token, device_type, device_info)

# 3. Eventually user logs out
device_mgr.on_logout(user_id, device_token, device_type)

## Real-World Example: Push Notification Preferences

Managing detailed push notification preferences per device.

In [ ]:
def update_push_preferences(client, user_id, device_token, device_type, preferences):
    """Update push notification preferences for a device."""
    
    # Build preference metadata
    metadata = {
        "push_enabled": preferences.get("enabled", True),
        "sound_enabled": preferences.get("sound", True),
        "vibration_enabled": preferences.get("vibration", True),
        "categories": preferences.get("categories", []),
        "quiet_hours": preferences.get("quiet_hours", {}),
        "frequency_cap": preferences.get("frequency_cap", "unlimited"),
        "preferences_updated_at": datetime.now().isoformat()
    }
    
    try:
        update_device(
            client,
            user_id,
            device_token,
            device_type,
            metadata
        )
        print("Push preferences updated successfully")
        return True
    except CustomerIOError as e:
        print(f"Failed to update preferences: {e}")
        return False

In [ ]:
# Example: Set detailed push preferences
user_preferences = {
    "enabled": True,
    "sound": True,
    "vibration": False,
    "categories": [
        "order_updates",
        "promotions",
        "account_alerts"
    ],
    "quiet_hours": {
        "enabled": True,
        "start": "22:00",
        "end": "08:00",
        "timezone": "America/Los_Angeles"
    },
    "frequency_cap": "5_per_day"
}

update_push_preferences(
    client,
    "user_prefs_001",
    "prefs_device_token",
    "ios",
    user_preferences
)

## Error Handling and Validation

Proper error handling for device operations.

In [ ]:
# Example: Validation errors
invalid_operations = [
    # Empty device token
    lambda: register_device(client, "user123", "", "ios"),
    # Invalid device type
    lambda: register_device(client, "user123", "token123", "windows"),
    # Invalid metadata type
    lambda: register_device(client, "user123", "token123", "ios", "not a dict"),
    # Missing user ID
    lambda: register_device(client, "", "token123", "android")
]

for i, operation in enumerate(invalid_operations):
    try:
        operation()
    except ValidationError as e:
        print(f"Validation error {i+1}: {e}")

## Device Token Rotation

Handle device token updates (common with iOS).

In [ ]:
def rotate_device_token(client, user_id, old_token, new_token, device_type):
    """Handle device token rotation."""
    
    try:
        # Delete old token
        delete_device(client, user_id, old_token, device_type)
        print(f"Removed old token: {old_token[:10]}...")
        
        # Register new token
        register_device(
            client,
            user_id,
            new_token,
            device_type,
            {"token_rotated_at": datetime.now().isoformat()}
        )
        print(f"Registered new token: {new_token[:10]}...")
        return True
        
    except CustomerIOError as e:
        print(f"Token rotation failed: {e}")
        return False

# Example usage
rotate_device_token(
    client,
    "user_rotation_001",
    "old_token_abc123",
    "new_token_xyz789",
    "ios"
)

## Best Practices

### Token Management
- Always validate tokens before registration
- Handle token rotation gracefully
- Clean up old/invalid tokens regularly

### Metadata Usage
- Track app version for targeted campaigns
- Store device capabilities (sound, vibration)
- Record user preferences per device
- Include timezone for optimal send times

### Security
- Never log full device tokens
- Validate tokens server-side
- Remove tokens on logout/uninstall

### Performance
- Update devices asynchronously when possible
- Batch device operations for multiple users
- Cache device states locally to minimize API calls

## Next Steps

Now that you understand device management, explore:

- **05_batch_operations.ipynb** - Bulk device registration and updates
- **02_event_tracking.ipynb** - Track device-related events
- **01_people_management.ipynb** - Link devices to user profiles